# Project Cyber-Trace: Gold Layer (Business Analytics)
**Author:** Jakub Milczarczyk
**Layer:** Gold (Aggregated/Curated)

## Objective
Create business-level aggregates to identify **Top Attackers** and **Daily Threat Trends**.
This layer feeds directly into BI Dashboards (Power BI / Databricks SQL).

## Logic
* Reads clean data from **Silver Layer**.
* Aggregates events by Date, Source IP, and Event Type.
* Calculates total occurrences (Attack Intensity).
* Overwrites the Gold Table (Full Refresh pattern for simplicity in MVP).

In [0]:
# ==============================================================================
# CELL 1: SETUP & AUTH
# ==============================================================================

from pyspark.sql import functions as F
from src.config import setup_authentication, ProjectConfig, Paths

setup_authentication(spark, dbutils)


In [0]:
# ==============================================================================
# CELL 2: READ SILVER DATA
# We read as a standard DataFrame (Batch Mode) for aggregation
# ==============================================================================

df_silver = spark.read.table("silver_security_logs")

In [0]:
# ==============================================================================
# CELL 3: BUSINESS LOGIC (AGGREGATION)
# ==============================================================================

df_staged = df_silver.withColumn("report_date", F.to_date("event_timestamp"))

df_gold = (df_staged
    .groupBy("report_date", "source_ip", "event_id", "hostname")
    .agg(
        F.count("*").alias("total_events"),
        F.first("process_image").alias("sample_process"),
        F.first("detection_tags").alias("tags")
    )
    .filter(F.col("source_ip").isNotNull())
    .orderBy(F.col("total_events").desc())          
)


In [0]:
# ==============================================================================
# CELL 4: WRITE TO GOLD (DELTA) AND REGISTER TEMP VIEW
# ==============================================================================

gold_name_view = "gold_threat_view"

table_path_gold = Paths.GOLD

# 1. Physically save the data to the table path

(df_gold.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .save(table_path_gold)
)

print(f"Gold Table saved physically: {table_path_gold}")

# 2. Create the temporary view

spark.read.format("delta").load(table_path_gold).createOrReplaceTempView(gold_name_view)

print(f"Temporary View {gold_name_view} created for visualization.")

In [0]:
%sql
SELECT
  source_ip as Attacker_IP,
  hostname as Victim_Host,
  sum(total_events) as Attack_Volume
FROM gold_threat_view
WHERE source_ip IS NOT NULL
GROUP BY source_ip, hostname
ORDER BY Attack_Volume DESC